In [1]:
import pandas as pd
from pprint import pprint
from pymongo import MongoClient
from pymongo.errors import BulkWriteError
from gensim.models import KeyedVectors
import re, pickle
from tqdm import tqdm


In [2]:
client = MongoClient('mongodb://localhost:27017/')

db = client.covid_hack
tweets_db = db.tweets


In [3]:
tweets_sent = db.tweets_for_sentiment

In [4]:
def clear_tweet_text(tweet_text):
    tweet_text = re.sub(r'http\S+', '', tweet_text)
    #tweet_text = re.sub(r'#\S+', '', tweet_text)
    #tweet_text = re.sub(r'@\S+', '', tweet_text)
    #tweet_text = re.sub(r'[^\w\s]',' ',tweet_text)
    return tweet_text

def lower_tr(text):
    lowerMap = {ord(u'A'): u'a',ord(u'A'): u'a',ord(u'B'): u'b',ord(u'C'): u'c',ord(u'Ç'): u'ç',ord(u'D'): u'd',ord(u'E'): u'e',ord(u'F'): u'f',ord(u'G'): u'g',ord(u'Ğ'): u'ğ',ord(u'H'): u'h',ord(u'I'): u'ı',ord(u'İ'): u'i',ord(u'J'): u'j',ord(u'K'): u'k',ord(u'L'): u'l',ord(u'M'): u'm',ord(u'N'): u'n',ord(u'O'): u'o',ord(u'Ö'): u'ö',ord(u'P'): u'p',ord(u'R'): u'r',ord(u'S'): u's',ord(u'Ş'): u'ş',ord(u'T'): u't',ord(u'U'): u'u',ord(u'Ü'): u'ü',ord(u'V'): u'v',ord(u'Y'): u'y',ord(u'Z'): u'z',ord(u'W'): u'w',ord(u'Q'): u'q',ord(u'X'): u'x'}
    text = text.translate(lowerMap)
    return text

things_to_remove = ['#şüpheduymuyorum', '#duysesimi', '#biralkısda', '#neyinnesi', 
                    "#biralkışda", '#hepsinitopla', '#kabuledilemez', '#gittimgeldim', 
                    '#işaş', '#veresiyedefteri', '#çağrımvar', '#cağrımvar', '#sesverin',
                   '@fatihportakal']
stop_words = []
with open('stop_words') as f:
    stop_words = f.read().split('\n')

In [5]:
tweet_data_all = []
error_counter = 0

counter = 0
for tweet in tqdm(tweets_db.find()):
    if tweet["full_text"].startswith('RT') or 'zemberek_nlp' not in tweet:
        continue
    try:
        counter += 1
        tweet_data = {}
        tweet_data['id'] = tweet['id']
        tweet_data['created_at'] = tweet['created_at']
        tweet_data['hashtag_id'] = tweet['hashtag_id']
        tweet_data['full_text'] = tweet['full_text']
        tweet_data['retweet_count'] = tweet['retweet_count']
        tweet_data['favorite_count'] = tweet['favorite_count']
        tweet_data['user_follower_count'] = tweet['user']['followers_count']
        tweet_data['mentions'] = tweet['zemberek_nlp']['mentions']
        tweet_data['hashtags'] = tweet['zemberek_nlp']['hashtags']
        tweet_data['lemmatized_text'] = clear_tweet_text(tweet['zemberek_nlp']['lemmatized_text']) + \
            ' ' + tweet_data['mentions'] + ' '  + tweet_data['hashtags']
        tweet_data['lemmatized_text_v2'] = clear_tweet_text(tweet['zemberek_nlp']['lemmatized_text_v2']) + \
            ' ' + tweet_data['mentions'] + ' '  + tweet_data['hashtags']
        # tweet_data['tokenized_text'] = clear_tweet_text(tweet['zemberek_nlp']['tokenized_text'])
        # tweet_data['num_of_hashtags'] = len(tweet['entities']['hashtags'])
        # tweet_data['num_of_mentions'] = len(tweet['entities']['user_mentions'])
        # tweet_data['num_of_urls'] = len(tweet['entities']['urls'])
        
        # Remove stops words and numbers
        processed_text_arr = []
        for word in tweet_data["lemmatized_text"].split():
            word = lower_tr(word)
            if word not in stop_words and word not in things_to_remove and "fox" not in word:
                try:
                    number = int(word)
                except Exception as e:
                    processed_text_arr.append(word)
        
        processed_text = " ".join(processed_text_arr)
        tweet_data['processed_text'] = processed_text
        
        processed_text_arr = []
        for word in tweet_data["lemmatized_text_v2"].split():
            word = lower_tr(word)
            if word not in stop_words and word not in things_to_remove and "fox" not in word:
                try:
                    number = int(word)
                except Exception as e:
                    processed_text_arr.append(word)
        
        processed_text = " ".join(processed_text_arr)
        tweet_data['processed_text_v2'] = processed_text
        
        if len(tweet_data['processed_text'].split()) <= 6:
            continue
        tweet_data_all.append(tweet_data)
    except KeyError as e: 
        print("Error on tweet: {}".format(tweet["full_text"]))
        error_counter += 1

print("Error: {}".format(error_counter))

135195it [00:46, 2883.37it/s]

Error: 0


### Fields To Add DataFrame
* id
* created_at
* hashtag_id
* full_text
* user.followers_count
* zemberek_nlp.lemmatized_text
* zemberek_nlp.tokenized_text

In [6]:
tweet_df = pd.DataFrame(tweet_data_all)

In [7]:
tweet_df.tail()

,id,created_at,hashtag_id,full_text,retweet_count,favorite_count,user_follower_count,mentions,hashtags,lemmatized_text,lemmatized_text_v2,processed_text,processed_text_v2
66429,1251177313636679681,Fri Apr 17 15:54:51 +0000 2020,#sesverin,#sesverin sesimizsin foxtv fati portakal iyi v...,0,1,5,,#sesverin,sesimiz fox fat portakal iyi var doğru ...,ses Fox Fat portakal iyi var doğru adres yanda...,sesimiz fat iyi doğru adreste yandaş deyili ca...,ses fat iyi doğru adres yandaş deyi candaş
66430,1251177308104396800,Fri Apr 17 15:54:50 +0000 2020,#sesverin,@fatihportakal Özel eğitim ve rehabilitasyon m...,0,0,29,@fatihportakal,#sesverin,özel eğitim ve rehabilitasyon merkez çalı...,özel eğitim ve rehabilitasyon merkez çalışmak ...,özel eğitim rehabilitasyon merkez çalışan işve...,özel eğitim rehabilitasyon merkez çalışmak işv...
66431,1251177187300061184,Fri Apr 17 15:54:21 +0000 2020,#sesverin,Sürekli önümüzdeki iki hafta çok önemli deniyo...,1,4,13712,,#sesverin,sürekli önümüzdeki iki hafta çok önemli ...,sürekli ön iki hafta çok önem denemek lastik g...,sürekli önümüzdeki hafta önemli dene lastik ha...,sürekli hafta önem denemek lastik hafta uzamak...
66432,1251177172984832002,Fri Apr 17 15:54:18 +0000 2020,#sesverin,"@fatihportakal Sn. @fatihportakal 8 aydır, 240...",1,0,142,@fatihportakal @fatihportakal @armağanportakal,#öğrenciaffi📚 #evdekal #sesverin #mühendismima...,sn 8 ay 240 gün yaz biz sadece okumak ...,Sn 8 ay 240 gün yazmak biz sadece okumak istem...,sn ay gün yaz sadece okumak iste lütfen sessiz...,sn ay gün yazmak sadece okumak istemek lütfen ...
66433,1251177107859931136,Fri Apr 17 15:54:02 +0000 2020,#sesverin,"Düşünce ve kanaat hürriyeti\nMadde 25: Herkes,...",0,9,225,@fatihportakal,#sesverin #akşam7,düşünce ve kanaat hürriyet madde 25 herk...,düşünce ve kanaat hürriyet madde 25 herkes düş...,düşünce kanaat hürriyet madde düşünce kanaat h...,düşünce kanaat hürriyet madde düşünce kanaat h...


In [8]:
tweet_df.describe()

,id,retweet_count,favorite_count,user_follower_count
count,6.643400e+04,66434.000000,66434.000000,6.643400e+04
mean,1.248502e+18,3.613752,12.350483,2.321682e+03
std,1.813423e+15,62.864753,463.816251,3.052767e+04
min,1.245388e+18,0.000000,0.000000,0.000000e+00
25%,1.247208e+18,0.000000,0.000000,1.000000e+01
50%,1.248284e+18,0.000000,1.000000,6.900000e+01
75%,1.250106e+18,0.000000,3.000000,3.250000e+02
max,1.253049e+18,10427.000000,105502.000000,3.403474e+06


In [9]:
tweet_df.to_csv('data/tweet_data.csv')

In [53]:
pd.set_option('display.max_colwidth', -1)

tweet_df.sample(n=5)

/home/kemal/.anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


,id,created_at,hashtag_id,full_text,retweet_count,favorite_count,user_follower_count,mentions,hashtags,lemmatized_text,lemmatized_text_v2,processed_text,processed_text_v2
43711,1249747656739602434,Mon Apr 13 17:13:55 +0000 2020,#gittimgeldim,Ptt çalışanlarına verilen ek ödeme sadece yönetim ve denetim kuruluna verildi alt kademe çalışanları olan dagiticalara neden ek ödeme verilmedi lütfen sesimizi duyurun lütfen 🙏 #gittimgeldim,0,0,34,,#gittimgeldim,ptt çalışan verilen ek ödeme sadece yönetim ve denetim kurul veril alt kademe çalışan olan dağıticalara neden ek ödem veril lütfen ses duyur lütfen 🙏 #gittimgeldim,Ptt çalışmak vermek ek ödemek sadece yönetim ve denetim kurul vermek alt kademe çalışmak olmak dağıticalara neden ek ödem vermek lütfen ses duyurmak lütfen 🙏 #gittimgeldim,ptt çalışan verilen ek ödeme sadece yönetim denetim kurul veril alt kademe çalışan dağıticalara ek ödem veril lütfen ses duyur lütfen 🙏,ptt çalışmak vermek ek ödemek sadece yönetim denetim kurul vermek alt kademe çalışmak dağıticalara ek ödem vermek lütfen ses duyurmak lütfen 🙏
20749,1247579851608870912,Tue Apr 07 17:39:50 +0000 2020,#biralkışda,@fatihportakal Tüm Türkiye aslında sana minnettar olup alkışlamalı FATIH\nİlk alkış Almanya'da ikamet eden tüm Cumhuriyetcilerin adına benden gelsin👍❤👍#biralkışda\n#FOXHaber,0,1,38,@fatihportakal,#foxhaber,tüm türkiye aslında sen minnettar olup alkışla fatih ilk alkış almanya ikamet eden tüm cumhuriyetçi ad ben gelsin👍❤👍#biralkışda @fatihportakal #foxhaber,tüm Türkiye aslında sen minnettar olmak alkışlamak Fatih ilk alkış Almanya ikamet etmek tüm cumhuriyet ad ben gelsin👍❤👍#biralkışda @fatihportakal #foxhaber,türkiye minnettar alkışla ilk alkış almanya ikamet cumhuriyetçi ad gelsin👍❤👍#biralkışda,türkiye minnettar alkışlamak ilk alkış almanya ikamet etmek cumhuriyet ad gelsin👍❤👍#biralkışda
27965,1247935271427100672,Wed Apr 08 17:12:08 +0000 2020,#hepsinitopla,Bizler Aile Hekimliğinde gruplandırma kapsamında çalışan sağlık çalışanlarıyız. ATT/HEMŞİRE/TIBBİ SEKRETER/EBE olarak görev yapıyoruz.Özverili çalışmalarımızın karşılığını alamıyorz.Kadro ve iş garantimiz bulunmamakta.KADRO talep ediyoruz.u\n#ASMsağlıkçılarınaKadro\n#hepsinitopla,0,1,31,,#aşmsağlıkçılarınakadro #hepsinitopla,biz aile hekimlik gruplandır kapsam çalışan sağlık çalışanları att hemşire tıbbi sekreter ebe olarak görev yap özverili çalışma karşılık al kadro ve iş garanti bulunmamak kadro talep et u #aşmsağlıkçılarınakadro #hepsinitopla,biz aile hekim grup kapsam çalışmak sağlık çalışmak att hemşire tıbbî sekreter ebe olmak görev yapmak özveri çalışmak karşılık almak kadro ve iş garanti bulunmak kadro talep etmek U #aşmsağlıkçılarınakadro #hepsinitopla,aile hekimlik gruplandır kapsam çalışan sağlık çalışanları att hemşire tıbbi sekreter ebe görev yap özverili çalışma karşılık al kadro iş garanti bulunmamak kadro talep et #aşmsağlıkçılarınakadro,aile hekim grup kapsam çalışmak sağlık çalışmak att hemşire tıbbî sekreter ebe görev özveri çalışmak karşılık almak kadro iş garanti bulunmak kadro talep etmek #aşmsağlıkçılarınakadro
34527,1248302882878824448,Thu Apr 09 17:32:54 +0000 2020,#kabuledilemez,İşveren işten çıkaramaz ama ücretsiz izine gönderebilir. Bunda işçiye ne fayda var.işten çıkarsa daha iyi. İşçi hem kıdem tazminatını alır hem de işsizlik sigortasından faydalanır. Aksi durumda günlük 39tlye mahkum olur. #kabuledilemez,0,0,1,,#kabuledilemez,işveren iş çıkar ama ücretsiz iz gönderebil bu işçi ne fayda var iş çık daha iyi işçi hem kıdem tazminat al hem de işsizlik sigorta faydalan aksi durum günlük 39tlye mahkum ol #kabuledilemez,işveren iş çıkarmak ama ücret iz göndermek bu işçi ne fayda var iş çıkmak daha iyi işçi hem kıdem tazminat almak hem de iş sigorta faydalanmak aksi durum günlük 39tlye mahkûm olmak #kabuledilemez,işveren iş çıkar ücretsiz iz gönderebil işçi fayda iş çık iyi işçi kıdem tazminat al işsizlik sigorta faydalan aksi durum günlük 39tlye mahkum ol,işveren iş çıkarmak ücret iz göndermek işçi fayda iş çıkmak iyi işçi kıdem t

In [48]:
tweets_db.find_one({'id': 1251197081211080706})

{'_id': ObjectId('5ea0aeccfbb6515a48f9aec9'),
 'created_at': 'Fri Apr 17 17:13:24 +0000 2020',
 'id': 1251197081211080706,
 'id_str': '1251197081211080706',
 'full_text': '@ercumentovali Hocam senin ilk önce saçı sakalı kesip badem bıyık bırakman lazım.yoksa yaptığın çalışmaların hiç bir önemi olmaz.#ebrusalli #vaka78546 #sesverin #BarisArduc',
 'truncated': False,
 'display_text_range': [15, 172],
 'entities': {'hashtags': [{'text': 'ebrusalli', 'indices': [129, 139]},
   {'text': 'vaka78546', 'indices': [140, 150]},
   {'text': 'sesverin', 'indices': [151, 160]},
   {'text': 'BarisArduc', 'indices': [161, 172]}],
  'symbols': [],
  'user_mentions': [{'screen_name': 'ercumentovali',
    'name': 'Ercüment Ovalı',
    'id': 952299252,
    'id_str': '952299252',
    'indices': [0, 14]}],
  'urls': []},
 'metadata': {'result_type': 'recent', 'iso_language_code': 'tr'},
 'source': '<a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>',
 'in_reply_to_status_i

In [ ]:
tweet_data_all = []
error_counter = 0

counter = 0
for tweet in tqdm(tweets_sent.find()):
    if tweet["full_text"].startswith('RT') or 'zemberek_nlp' not in tweet:
        continue
    try:
        counter += 1
        tweet_data = {}
        tweet_data['id'] = tweet['id']
        tweet_data['created_at'] = tweet['created_at']
        tweet_data['hashtag_id'] = tweet['hashtag_id']
        tweet_data['full_text'] = tweet['full_text']
        tweet_data['retweet_count'] = tweet['retweet_count']
        tweet_data['favorite_count'] = tweet['favorite_count']
        tweet_data['user_follower_count'] = tweet['user']['followers_count']
        tweet_data['mentions'] = tweet['zemberek_nlp']['mentions']
        tweet_data['hashtags'] = tweet['zemberek_nlp']['hashtags']
        tweet_data['lemmatized_text'] = clear_tweet_text(tweet['zemberek_nlp']['lemmatized_text']) + \
            ' ' + tweet_data['mentions'] + ' '  + tweet_data['hashtags']
        tweet_data['lemmatized_text_v2'] = clear_tweet_text(tweet['zemberek_nlp']['lemmatized_text_v2']) + \
            ' ' + tweet_data['mentions'] + ' '  + tweet_data['hashtags']
        # tweet_data['tokenized_text'] = clear_tweet_text(tweet['zemberek_nlp']['tokenized_text'])
        # tweet_data['num_of_hashtags'] = len(tweet['entities']['hashtags'])
        # tweet_data['num_of_mentions'] = len(tweet['entities']['user_mentions'])
        # tweet_data['num_of_urls'] = len(tweet['entities']['urls'])
        
        # Remove stops words and numbers
        processed_text_arr = []
        for word in tweet_data["lemmatized_text"].split():
            word = lower_tr(word)
            if word not in stop_words and word not in things_to_remove and "fox" not in word:
                try:
                    number = int(word)
                except Exception as e:
                    processed_text_arr.append(word)
        
        processed_text = " ".join(processed_text_arr)
        tweet_data['processed_text'] = processed_text
        
        processed_text_arr = []
        for word in tweet_data["lemmatized_text_v2"].split():
            word = lower_tr(word)
            if word not in stop_words and word not in things_to_remove and "fox" not in word:
                try:
                    number = int(word)
                except Exception as e:
                    processed_text_arr.append(word)
        
        processed_text = " ".join(processed_text_arr)
        tweet_data['processed_text_v2'] = processed_text
        
        if len(tweet_data['processed_text'].split()) <= 6:
            continue
        tweet_data_all.append(tweet_data)
    except KeyError as e: 
        print("Error on tweet: {}".format(tweet["full_text"]))
        error_counter += 1

print("Error: {}".format(error_counter))